In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
train_1 = pd.read_csv("f://watermelondata//Watermelon-train1.csv",encoding = "gbk")
test_1 = pd.read_csv("f://watermelondata//Watermelon-test1.csv",encoding = "gbk")

In [3]:
train_1 = train_1.drop(columns=['编号'],axis=0)
test_1 = test_1.drop(columns=['编号'],axis=0)

In [4]:
train_1.head()

,色泽,根蒂,敲声,纹理,好瓜
0,青绿,蜷缩,浊响,清晰,是
1,乌黑,蜷缩,沉闷,清晰,是
2,乌黑,蜷缩,浊响,清晰,是
3,青绿,蜷缩,沉闷,清晰,是
4,浅白,蜷缩,浊响,清晰,是


In [5]:
##id3决策树
class Node:#the node of the decision tree
    Ai = None
    Avalue = None           #value
    child_list = []          #children
    class_y = None           #classification
    print_content = None
    best_gain = None

class DecisionTree:
    root = None
    labels = []
    attr_value_set = {}

    def __init__(self, data, labels):
        self.root = Node()
        self.labels = labels
        for label in self.labels:
            col_num = labels.index(label)
            col = [a[col_num] for a in data]
            self.attr_value_set[label] = set(col)

    def calculate_entropy(self, data):
        # 数据量
        row_num, col_num = np.shape(data)
        label_count = {}
        for row in data:
            current_label = row[-1]
            if current_label not in label_count.keys():
                label_count[current_label] = 0
            label_count[current_label] += 1
        entropy = 0
        # 计算类别的熵
        for key in label_count:
            prob = float(label_count[key]/row_num)
            entropy -= prob*math.log2(prob)
        return entropy

    # 找得类别中最多的
    def Main(self, class_y):
        y_count = {}
        for y in class_y:
            if y not in y_count.keys():
                y_count[y] = 0
            y_count[y] += 1
        max_count = 0
        max_key = class_y[0]
        for key in y_count:
            if y_count[key] > max_count:
                max_count = y_count[key]
                max_key = key
        return max_key

    # 划分数据集，根据属性ai,获取值为ai_v的数据集data_v
    # i：第i个属性（这里data第i列）
    # v: ai第v个取值
    def SplitData(self, data, i, v):
        data_v = []
        for row in data:
            if row[i] == v:
                reduced_row = row[:i]
                reduced_row.extend(row[i + 1:])
                data_v.append(reduced_row)
        return data_v

    # 最优划分属性，根据shannon entropy & 信息增益
    def find_best_attr(self, data):
        attr_num = len(data[0]) - 1
        ent_d = self.calculate_entropy(data)
        # 增益
        best_gain = 0
        # 最优属性序号(0~len(data)-1)
        best_attr = -1
        for i in range(attr_num):
            # 属性ai所有取值，data第i列
            attr_i = [a[i] for a in data]
            # 属性ai所有取值
            attr_iv_set = set(attr_i)

            # 属性ai的v个取值的熵求和(例：ai表示色泽，ai_v=青绿，v=0,1,2...)
            # sum: |D^v|/|D|*ent(D^v)(v=0,1,2...)
            sum_ent_dv = 0
            for attr_iv in attr_iv_set:
                data_v = self.SplitData(data, i, attr_iv)
                prob = len(data_v)/len(data)
                sum_ent_dv += prob*self.calculate_entropy(data_v)
            gain_v = ent_d - sum_ent_dv
            # 获取最大增益和对应属性
            if gain_v > best_gain:
                best_gain = gain_v
                best_attr = i
        return [best_attr, best_gain]

    def create_tree(self, data, labels, node=None):
        if node is None:
            node = self.root
        # get class
        class_y = [cls[-1] for cls in data]
        if class_y.count(class_y[0]) == len(class_y):
            node.class_y = class_y[0]
            return
        # attribute none
        if labels is None or len(labels) == 0:
            node.class_y = self.Main(class_y)
            return
        # 从属性中找到最优划分属性(ID3算法)
        best_attr, best_gain = self.find_best_attr(data)
        node.Ai = labels[best_attr]
        node.best_gain = best_gain
        # 最优属性ai所有样本值，data第i列
        # 最优属性ai所有取值
        attr_iv_set = self.attr_value_set[node.Ai]
        for attr_iv in attr_iv_set:
            # 为node生成一个分支节点，data_v表示data在属性attr_i取值为attr_iv的样本集合
            child_node = Node()
            child_node.child_list = []
            child_node.print_content = str(node.Ai)+str(attr_iv)
            child_node.Avalue = attr_iv
            node.child_list.append(child_node)
            data_v = self.SplitData(data, best_attr, attr_iv)
            # data_v是空，标记为叶子节点，分类属于data中节点最多的
            if data_v is None or len(data_v) == 0:
                class_v_y = [cls[-1] for cls in data]
                child_node.class_y = self.Main(class_v_y)
            else:
                label_v = labels[:best_attr] + labels[best_attr + 1:]
                self.create_tree(data_v, label_v, child_node)
        return

    # 给定属性，进行预测，树前序遍历
    def predict(self, x, node=None):
        if node is None:
            node = self.root
        if node.class_y is not None:
            print(node.class_y)
            return node
        # 节点对应属性位置
        ai_index = self.labels.index(node.Ai)
        ai_v = x[ai_index]
        print(node.Ai)
        if len(node.child_list) > 0:
            for child_node in node.child_list:
                leaf = None
                if child_node.Avalue == ai_v:
                    print(child_node.print_content)
                    leaf = self.predict(x, child_node)
                if leaf is not None:
                    return leaf


    def bfs_tree(self):
        queue = []
        if self.root is not None:
            queue.append(self.root)
        while queue is not None and len(queue) > 0:
            # 每层节点数
            level_num = len(queue)
            for i in range(level_num):
                if len(queue[0].child_list) > 0:
                    for node in queue[0].child_list:
                        queue.append(node)
                print_content = queue[0].print_content if queue[0].print_content is not None else ""
                if queue[0].Ai is not None:
                    print(print_content, queue[0].Ai, queue[0].best_gain, end=' ')
                    print(",", end=' ')
                if queue[0].class_y is not None:
                    print(print_content, queue[0].class_y, end=' ')
                    print(",", end=' ')
                del queue[0]
            print(" ")



In [6]:
data_set = np.array(train_1).tolist()
test_set = np.array(test_1).tolist()

In [7]:
data_set

[['青绿', '蜷缩', '浊响', '清晰', '是'],
 ['乌黑', '蜷缩', '沉闷', '清晰', '是'],
 ['乌黑', '蜷缩', '浊响', '清晰', '是'],
 ['青绿', '蜷缩', '沉闷', '清晰', '是'],
 ['浅白', '蜷缩', '浊响', '清晰', '是'],
 ['青绿', '稍蜷', '浊响', '清晰', '是'],
 ['乌黑', '稍蜷', '浊响', '稍糊', '是'],
 ['乌黑', '稍蜷', '浊响', '清晰', '是'],
 ['乌黑', '稍蜷', '沉闷', '稍糊', '否'],
 ['青绿', '硬挺', '清脆', '清晰', '否'],
 ['浅白', '硬挺', '清脆', '模糊', '否'],
 ['浅白', '蜷缩', '浊响', '模糊', '否'],
 ['青绿', '稍蜷', '浊响', '稍糊', '否'],
 ['浅白', '稍蜷', '沉闷', '稍糊', '否'],
 ['浅白', '蜷缩', '浊响', '模糊', '否'],
 ['青绿', '蜷缩', '沉闷', '稍糊', '否']]

In [10]:
labels=['色泽','根蒂','敲声','纹理']

In [8]:
test_set

[['浅白', '蜷缩', '浊响', '清晰', '是'],
 ['乌黑', '稍蜷', '沉闷', '清晰', '是'],
 ['乌黑', '蜷缩', '沉闷', '清晰', '是'],
 ['青绿', '蜷缩', '沉闷', '稍糊', '是'],
 ['浅白', '蜷缩', '浊响', '清晰', '是'],
 ['青绿', '稍蜷', '浊响', '清晰', '否'],
 ['乌黑', '稍蜷', '浊响', '稍糊', '否'],
 ['青绿', '稍蜷', '浊响', '模糊', '否'],
 ['乌黑', '稍蜷', '沉闷', '稍糊', '否'],
 ['青绿', '硬挺', '清脆', '模糊', '否']]

In [11]:
tree = DecisionTree(data_set, labels)
tree.create_tree(data_set, labels)  # 输出决策树模型结果
tree.bfs_tree()


 纹理 0.5026152487479011 ,  
纹理清晰 根蒂 0.5435644431995964 , 纹理模糊 否 , 纹理稍糊 色泽 0.3219280948873623 ,  
根蒂稍蜷 是 , 根蒂硬挺 否 , 根蒂蜷缩 是 , 色泽乌黑 敲声 1.0 , 色泽青绿 否 , 色泽浅白 否 ,  
敲声浊响 是 , 敲声清脆 是 , 敲声沉闷 否 ,  


In [18]:
for i in range(10):
    node = tree.predict(test_set[i])
    print(test_set[i][4])

纹理
纹理清晰
根蒂
根蒂蜷缩
是
是
纹理
纹理清晰
根蒂
根蒂稍蜷
是
是
纹理
纹理清晰
根蒂
根蒂蜷缩
是
是
纹理
纹理稍糊
色泽
色泽青绿
否
是
纹理
纹理清晰
根蒂
根蒂蜷缩
是
是
纹理
纹理清晰
根蒂
根蒂稍蜷
是
否
纹理
纹理稍糊
色泽
色泽乌黑
敲声
敲声浊响
是
否
纹理
纹理模糊
否
否
纹理
纹理稍糊
色泽
色泽乌黑
敲声
敲声沉闷
否
否
纹理
纹理模糊
否
否


In [15]:
class Node:#the node of the decision tree
    Ai = None
    Avalue = None           #value
    child_list = []          #children
    class_y = None           #classification
    print_content = None
    best_gain = None

class DecisionTree:
    root = None
    labels = []
    attr_value_set = {}

    def __init__(self, data, labels):
        self.root = Node()
        self.labels = labels
        for label in self.labels:
            col_num = labels.index(label)
            col = [a[col_num] for a in data]
            self.attr_value_set[label] = set(col)

    def calculate_entropy(self, data):
        # 数据量
        row_num, col_num = np.shape(data)
        label_count = {}
        for row in data:
            current_label = row[-1]
            if current_label not in label_count.keys():
                label_count[current_label] = 0
            label_count[current_label] += 1
        entropy = 0
        # 计算类别的熵
        for key in label_count:
            prob = float(label_count[key]/row_num)
            entropy -= prob*math.log2(prob)
        return entropy

        return entropy
    # 找得类别中最多的
    def Main(self, class_y):
        y_count = {}
        for y in class_y:
            if y not in y_count.keys():
                y_count[y] = 0
            y_count[y] += 1
        max_count = 0
        max_key = class_y[0]
        for key in y_count:
            if y_count[key] > max_count:
                max_count = y_count[key]
                max_key = key
        return max_key

    # 划分数据集，根据属性ai,获取值为ai_v的数据集data_v
    # i：第i个属性（这里data第i列）
    # v: ai第v个取值
    def SplitData(self, data, i, v):
        data_v = []
        for row in data:
            if row[i] == v:
                reduced_row = row[:i]
                reduced_row.extend(row[i + 1:])
                data_v.append(reduced_row)
        return data_v

    def find_best_attr(self, data):
        attr_num = len(data[0]) - 1
        ent_d = self.calculate_entropy(data)
        # 增益
        best_gain = 0
        # 最优属性序号(0~len(data)-1)
        best_attr = -1
        for i in range(attr_num):
            # 属性ai所有取值，data第i列
            attr_i = [a[i] for a in data]
            # 属性ai所有取值
            attr_iv_set = set(attr_i)

            # 属性ai的v个取值的熵求和
            sum_ent_dv = 0
            intr = 0
            for attr_iv in attr_iv_set:
                data_v = self.SplitData(data, i, attr_iv)
                prob = len(data_v)/len(data)
                intr -= prob*math.log2(prob)
                print(intr)
                sum_ent_dv += prob*self.calculate_entropy(data_v)
            gain_v = ent_d - sum_ent_dv
            IGRv = 0
            if(intr!=0):
                IGRv = gain_v/intr
            # 获取最大增益和对应属性
            if IGRv > best_gain:
                best_gain = IGRv
                best_attr = i
        return [best_attr, best_gain]

    def create_tree(self, data, labels, node=None):
        # 初始化根节点
        if node is None:
            node = self.root
        # get class
        class_y = [cls[-1] for cls in data]
        # 分类相同，标记为叶子节点
        if class_y.count(class_y[0]) == len(class_y):
            node.class_y = class_y[0]
            return
        # attribute none
        if labels is None or len(labels) == 0:
            node.class_y = self.Main(class_y)
            return
        # 从属性中找到最优划分属性
        best_attr, best_gain = self.find_best_attr(data)
        node.Ai = labels[best_attr]
        node.best_gain = best_gain
        attr_iv_set = self.attr_value_set[node.Ai]
        for attr_iv in attr_iv_set:
            child_node = Node()
            child_node.child_list = []
            child_node.print_content = str(node.Ai)+str(attr_iv)
            child_node.Avalue = attr_iv
            node.child_list.append(child_node)
            data_v = self.SplitData(data, best_attr, attr_iv)
            if data_v is None or len(data_v) == 0:
                class_v_y = [cls[-1] for cls in data]
                child_node.class_y = self.Main(class_v_y)
            else:
                label_v = labels[:best_attr] + labels[best_attr + 1:]
                self.create_tree(data_v, label_v, child_node)
        return

    # 给定属性，进行预测，树前序遍历
    def predict(self, x, node=None):
        if node is None:
            node = self.root
        if node.class_y is not None:
            print(node.class_y)
            return node
        # 节点对应属性位置
        ai_index = self.labels.index(node.Ai)
        ai_v = x[ai_index]
        print(node.Ai)
        if len(node.child_list) > 0:
            for child_node in node.child_list:
                leaf = None
                if child_node.Avalue == ai_v:
                    print(child_node.print_content)
                    leaf = self.predict(x, child_node)
                if leaf is not None:
                    return leaf


    def bfs_tree(self):
        queue = []
        if self.root is not None:
            queue.append(self.root)
        while queue is not None and len(queue) > 0:
            # 每层节点数
            level_num = len(queue)
            for i in range(level_num):
                if len(queue[0].child_list) > 0:
                    for node in queue[0].child_list:
                        queue.append(node)
                print_content = queue[0].print_content if queue[0].print_content is not None else ""
                if queue[0].Ai is not None:
                    print(print_content, queue[0].Ai, queue[0].best_gain, end=' ')
                    print(",", end=' ')
                if queue[0].class_y is not None:
                    print(print_content, queue[0].class_y, end=' ')
                    print(",", end=' ')
                del queue[0]



In [16]:
train_2 = pd.read_csv("f://watermelondata//Watermelon-train2.csv",encoding = "gbk")
test_2 = pd.read_csv("f://watermelondata//Watermelon-test2.csv",encoding = "gbk")

In [19]:
train_2 = train_2.sort_values(by=['密度'],ascending=True)

In [20]:
train_2 = train_2.drop(columns=['编号'],axis=0)
test_2 = test_2.drop(columns=['编号'],axis=0)

In [30]:
test_2

,色泽,根蒂,敲声,纹理,密度,好瓜
0,乌黑,稍蜷,浊响,清晰,0.403,是
1,青绿,稍蜷,浊响,稍糊,0.481,是
2,乌黑,稍蜷,浊响,清晰,0.337,是
3,乌黑,稍蜷,沉闷,稍糊,0.666,否
4,青绿,硬挺,清脆,清晰,0.243,否


In [31]:
data_set2 = np.array(train_2).tolist()
test_set2 = np.array(test_2).tolist()

In [23]:
for i in range(train_2.shape[0]//3):
    data_set2[i][4]=str(data_set2[train_2.shape[0]//3][4])
for i in range(train_2.shape[0]//3,train_2.shape[0]*2//3):
    data_set2[i][4]=str(data_set2[(train_2.shape[0]*2)//3][4])
for i in range(train_2.shape[0]*2//3,train_2.shape[0]):
    data_set2[i][4]=str(data_set2[train_2.shape[0]-1][4])

In [24]:
data_set2

[['青绿', '硬挺', '清脆', '清晰', '0.437', '否'],
 ['浅白', '硬挺', '清脆', '模糊', '0.437', '否'],
 ['浅白', '蜷缩', '浊响', '模糊', '0.437', '否'],
 ['乌黑', '稍蜷', '浊响', '清晰', '0.437', '否'],
 ['青绿', '稍蜷', '浊响', '清晰', '0.437', '是'],
 ['乌黑', '稍蜷', '浊响', '清晰', '0.639', '是'],
 ['乌黑', '稍蜷', '浊响', '稍糊', '0.639', '是'],
 ['浅白', '蜷缩', '浊响', '清晰', '0.639', '是'],
 ['浅白', '蜷缩', '浊响', '模糊', '0.639', '否'],
 ['青绿', '蜷缩', '沉闷', '清晰', '0.639', '是'],
 ['乌黑', '蜷缩', '浊响', '清晰', '0.639', '是'],
 ['青绿', '稍蜷', '浊响', '稍糊', '0.774', '否'],
 ['浅白', '稍蜷', '沉闷', '稍糊', '0.774', '否'],
 ['乌黑', '稍蜷', '沉闷', '稍糊', '0.774', '否'],
 ['青绿', '蜷缩', '浊响', '清晰', '0.774', '是'],
 ['青绿', '蜷缩', '沉闷', '稍糊', '0.774', '否'],
 ['乌黑', '蜷缩', '沉闷', '清晰', '0.774', '是']]

In [44]:
test_set2

[['乌黑', '稍蜷', '浊响', '清晰', 0.437, '是'],
 ['青绿', '稍蜷', '浊响', '稍糊', 0.639, '是'],
 ['乌黑', '稍蜷', '浊响', '清晰', 0.437, '是'],
 ['乌黑', '稍蜷', '沉闷', '稍糊', 0.774, '否'],
 ['青绿', '硬挺', '清脆', '清晰', 0.437, '否']]

In [33]:
for i in range(5):
    if(test_set2[i][4]<0.437):
        test_set2[i][4] = 0.437
    elif(test_set2[i][4]>0.437 and test_set2[i][4]<0.639):
        test_set2[i][4] = 0.639
    elif(test_set2[i][4]>0.639):
        test_set2[i][4] = 0.774

In [35]:
label2=['色泽','根蒂','敲声','纹理','密度']
tree = DecisionTree(data_set2, label2)
tree.create_tree(data_set2, label2)  # 输出决策树模型结果
tree.bfs_tree()


0.5302942378338293
1.0605884756676587
1.5798634010685344
0.5271032608440674
0.8903341833441074
1.402081402756032
0.45031455668410414
0.8135454791841441
1.3328204045850196
0.4857553269571908
0.9273730341093995
1.4466479595102752
0.24043899066178465
0.4808779813235693
0.7213169719853539
0.9617559626471386
1.2021949533089233
1.442633943970708
1.6830729346324929
1.9235119252942776
2.1639509159560624
2.404389906617847
2.644828897279632
2.8852678879414166
3.1257068786032014
3.366145869264986
3.606584859926771
3.8470238505885557
4.08746284125034
0.5199666673076944
1.0399333346153887
1.3921472236645345
0.5283208335737187
0.8805347226228646
1.3516441151533924
0.38997500048077083
0.7421888895299167
1.224394445405986
0.3522138890491458
0.7044277780982916
1.0566416671474375
1.4088555561965832
1.761069445245729
2.113283334294875
2.4654972233440207
2.8177111123931664
3.169925001442312
0.38997500048077083
0.9182958340544896
0.0
0.5283208335737187
1.0566416671474375
1.584962500721156
0.528771237954944

In [47]:
for i in range(4):
    node = tree.predict(test_set2[i])
    print(test_set2[i][5])

纹理
纹理清晰
根蒂
根蒂稍蜷
密度
密度0.437
是
是
纹理
纹理稍糊
敲声
敲声浊响
色泽
色泽青绿
否
是
纹理
纹理清晰
根蒂
根蒂稍蜷
密度
密度0.437
是
是
纹理
纹理稍糊
敲声
敲声沉闷
否
否


In [45]:
tree.predict(test_set2[1])

纹理
纹理稍糊
敲声
敲声浊响
色泽
色泽青绿
否


In [46]:
test_set2[4][4]

0.437